In [2]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut

import numpy as np
import pandas as pd
import random
import math
from tqdm.auto import tqdm

In [3]:
dfX = pd.read_csv("data/X.csv", index_col=0)
X = dfX.values
dfY = pd.read_csv("data/y.csv", index_col=0)
y = dfY["class"].values

In [4]:
# These are the best hyperparameters
classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', 
                                    criterion='entropy', max_depth=20, max_features=None,
                                    min_samples_leaf=1, min_samples_split=5, n_estimators=100)

In [5]:
_ = classifier.fit(X, y)

In [6]:
probs = pd.Series(classifier.predict_proba(X)[:, 0], name="class_prob")
young = dfX["Age"] <= 25
woman = dfX["Sex"] == 1
foreign = dfX["foreign worker"] == 1
credit = 1 - dfY["class"]
table = pd.concat([young, woman, foreign, credit, probs], axis=1)

In [7]:
def get_group_ratio(labels):
    print(labels.sum())
    print(f"ratio young to old:\t{(labels[young].sum() / labels[young].size) / (labels[~young].sum() / labels[~young].size)}")
    print(f"ratio female to male:\t{(labels[woman].sum() / labels[woman].size) / (labels[~woman].sum() / labels[~woman].size)}")
    print(f"ratio foreign to else:\t{(labels[foreign].sum() / labels[foreign].size) / (labels[~foreign].sum() / labels[~foreign].size)}")

In [8]:
def adjust(criterion, labels, probs, p=1):
    ratio = labels.sum() / labels.size
    ratio_True = labels[criterion].sum() / labels[criterion].size
    ratio_False = labels[~criterion].sum() / labels[~criterion].size
    if ratio_True > ratio_False:
        return adjust(~criterion, labels, probs)
    # ratio = (ratio_True + ratio_False) / 2
    n_True = round(labels[criterion].size * (ratio - ratio_True) * p)
    n_False = round(labels[~criterion].size * (ratio_False - ratio) * p)
    changed_True = labels.sort_values(key=lambda _: probs, ascending = False)[criterion & (labels == False)][:n_True].replace(0, value=1)
    changed_False = labels.sort_values(key=lambda _: probs, ascending = True)[(~criterion) & (labels == True)][:n_False].replace(1, value=0)
    labels.update(changed_True)
    labels.update(changed_False)

In [9]:
credit = 1 - dfY["class"]
print("== No adjustment ==")
get_group_ratio(credit)
print(f"ratio overall: \t\t{credit.sum() / credit.size}")
n = 100
for i in range(n):
    criteria = [young, woman, foreign]
    random.shuffle(criteria)
    for criterion in criteria:
        adjust(criterion, credit, probs, p=i/n)
    # print(f"Adjustment round {i}")
print("== With Adjustment ==")
get_group_ratio(credit)
print(f"ratio overall: \t\t{credit.sum() / credit.size}")

== No adjustment ==
700
ratio young to old:	0.7948260481712757
ratio female to male:	0.8965673282047968
ratio foreign to else:	0.7765820195726738
ratio overall: 		0.7
== With Adjustment ==
699
ratio young to old:	0.9193592677345538
ratio female to male:	1.0020746887966805
ratio foreign to else:	0.8876714290829664
ratio overall: 		0.699


In [171]:
(1 - credit).to_csv("data/y_adjusted.csv")

In [186]:
loo = LeaveOneOut()
cv_results = cross_val_predict(classifier, X, (1 - credit).values, cv=loo)
# save to csv to avoid re-run
pd.DataFrame(cv_results).to_csv('cross_val_predictions_adjusted.csv')

In [14]:
# get class probabilities with cross-validation

loo = LeaveOneOut()
cv_results = cross_val_predict(classifier, X, (1 - credit).values, cv=loo, method='predict_proba')
# save to csv to avoid re-run
pd.DataFrame(cv_results).to_csv('cross_val_predictions_probabilities_adjusted.csv')

In [187]:
print('Test accuracy:')
print(accuracy_score(y, cv_results))

Test accuracy:
0.776


In [20]:
# Re-read X to get column names
X = pd.read_csv("data/X.csv", index_col=0)
# Flip y indexing: 1 = credit granted, 0 = no credit granted
cv_results = pd.read_csv('cross_val_predictions_adjusted.csv').iloc[:,1]
pred = np.array(pd.DataFrame(cv_results).replace({0:1, 1:0}))

In [77]:
# Implement group fairness measures (assume women being inferiorly treated to men,
# immigrant inferiorly treated to nonimmigrant, and young to old (split at 25)).

def group_fairness(group_split_condition, predictions = pred):
    discr_group = X[group_split_condition].index
    other_group = X[~group_split_condition].index

    # granted credit percentage for the discriminated
    discr_group_rate = predictions[discr_group].sum() / predictions[discr_group].size
    other_group_rate = predictions[other_group].sum() / predictions[other_group].size
    print('Discriminated group credit grant rate: {}'.format(discr_group_rate))
    print('Other group credit grant rate: {}'.format(other_group_rate))
    print('Difference: {}'.format(other_group_rate-discr_group_rate))
    print('Ratio: {}'.format(discr_group_rate/other_group_rate))
    print()

def check_intersectional_fairness(predictions = pred):
    female_condition = X['Sex'] == 1
    immigrant_condition = X['foreign worker'] == 1
    young_condition = X['Age'] <= 25

    print('Women - men')
    group_fairness(female_condition, predictions)

    # immigrant - not immigrant
    print('Immigrant - non immigrant')
    group_fairness(immigrant_condition, predictions)

    # young - old
    print('Young - old')
    group_fairness(young_condition, predictions)

    # Intersection groups

    # woman immigrant - other
    print('Woman immigrant - other')
    group_fairness(female_condition & immigrant_condition, predictions)

    # young immigrant - other
    print('Young immigrant - other')
    group_fairness(young_condition & immigrant_condition, predictions)

    # young woman - other
    print('Young woman - other')
    group_fairness(young_condition & female_condition, predictions)

    # young woman immigrant - other
    print('Young woman immigrant - other')
    group_fairness(female_condition & young_condition & immigrant_condition, predictions)

check_intersectional_fairness()

Women - men
Discriminated group credit grant rate: 0.7387096774193549
Other group credit grant rate: 0.755072463768116
Difference: 0.016362786348761094
Ratio: 0.9783295152002972

Immigrant - non immigrant
Discriminated group credit grant rate: 0.7455867082035307
Other group credit grant rate: 0.8648648648648649
Difference: 0.11927815666133423
Ratio: 0.8620846313603323

Young - old
Discriminated group credit grant rate: 0.7052631578947368
Other group credit grant rate: 0.7604938271604939
Difference: 0.05523066926575704
Ratio: 0.9273752563226246

Woman immigrant - other
Discriminated group credit grant rate: 0.7392739273927392
Other group credit grant rate: 0.7546628407460545
Difference: 0.01538891335331527
Ratio: 0.979608226982394

Young immigrant - other
Discriminated group credit grant rate: 0.7058823529411765
Other group credit grant rate: 0.7601476014760148
Difference: 0.05426524853483827
Ratio: 0.9286122215876642

Young woman - other
Discriminated group credit grant rate: 0.7142857

In [94]:
# output flipping

# 0. find output class probability for all
result_probabilities = pd.read_csv('cross_val_predictions_probabilities_adjusted.csv').iloc[:,1]
previous_classification = pd.read_csv('cross_val_predictions_adjusted.csv').iloc[:,1]

# 1. divide people into groups with different disadvantages
X
female_condition = X['Sex'] == 1
immigrant_condition = X['foreign worker'] == 1
young_condition = X['Age'] <= 25

# 3 disadvantages
female_young_immigrant = X[female_condition & immigrant_condition & young_condition].index

# 2 disadvantages
male_young_immigrant = X[~female_condition & immigrant_condition & young_condition].index
female_young_nonimmigrant = X[female_condition & ~immigrant_condition & young_condition].index
female_old_immigrant = X[female_condition & immigrant_condition & ~young_condition].index

# 1 disadvantage
female_old_nonimmigrant = X[female_condition & ~immigrant_condition & ~young_condition].index
male_young_nonimmigrant = X[~female_condition & ~immigrant_condition & young_condition].index
male_old_immigrant = X[~female_condition & immigrant_condition & ~young_condition].index

# no disadvantages
male_old_nonimmigrant = X[~female_condition & ~immigrant_condition & ~young_condition].index

# double check: no duplicates in groups: groups sum to 1000
female_young_immigrant.size + male_young_immigrant.size + female_young_nonimmigrant.size + female_old_immigrant.size + female_old_nonimmigrant.size + male_young_nonimmigrant.size + male_old_immigrant.size + male_old_nonimmigrant.size

# 2. determine ratio
ratio = 0.7 # in the training data, 70% were granted credit. enforce this.

# 3. order each group by class probability & change grant decisions

groups = [female_young_immigrant, male_young_immigrant, female_young_nonimmigrant, female_old_immigrant, female_old_nonimmigrant,
           male_young_nonimmigrant, male_old_immigrant,male_old_nonimmigrant]
previously_granted = previous_classification[previous_classification == 1].index # to check how many were flipped
grant_adjusted = pd.Series(np.ones(y.size, dtype=int))

for group in groups:
    credit_probabilities = result_probabilities[group]
    ordered = credit_probabilities.sort_values(ascending=False)
    last_index = round(ordered.size * ratio)
    grant = ordered[0:last_index].index
    grant_adjusted[grant] = 0
    print('\nSize of group: {}'. format(ordered.size))
    group_grants = pd.Series(np.ones(ordered.size, dtype=int), index=group)
    group_grants[grant] = 0

    print('Count of flipped labels: {}'.format(ordered.size - (group_grants == previous_classification[group]).sum()))


print()
print('Total count of flipped labels:')
print(y.size - (grant_adjusted == previous_classification).sum())
print()

print('Accuracy after final adjustment:')
print(accuracy_score(y, grant_adjusted))


Size of group: 104
Count of flipped labels: 9

Size of group: 83
Count of flipped labels: 10

Size of group: 1
Count of flipped labels: 0

Size of group: 199
Count of flipped labels: 19

Size of group: 6
Count of flipped labels: 2

Size of group: 2
Count of flipped labels: 0

Size of group: 577
Count of flipped labels: 48

Size of group: 28
Count of flipped labels: 6

Total count of flipped labels:
94

Accuracy after final adjustment:
0.764


In [83]:
print('Final group fairness\n')
check_intersectional_fairness(grant_adjusted.replace({0:1, 1:0}))

Final group fairness

Women - men
Discriminated group credit grant rate: 0.7
Other group credit grant rate: 0.7
Difference: 0.0
Ratio: 1.0

Immigrant - non immigrant
Discriminated group credit grant rate: 0.6998961578400831
Other group credit grant rate: 0.7027027027027027
Difference: 0.0028065448626196643
Ratio: 0.9960060707724259

Young - old
Discriminated group credit grant rate: 0.7
Other group credit grant rate: 0.7
Difference: 0.0
Ratio: 1.0

Woman immigrant - other
Discriminated group credit grant rate: 0.6996699669966997
Other group credit grant rate: 0.7001434720229556
Difference: 0.00047350502625587154
Ratio: 0.9993237028620895

Young immigrant - other
Discriminated group credit grant rate: 0.7005347593582888
Other group credit grant rate: 0.6998769987699877
Difference: -0.0006577605883011373
Ratio: 1.000939823125288

Young woman - other
Discriminated group credit grant rate: 0.7047619047619048
Other group credit grant rate: 0.6994413407821229
Difference: -0.00532056397978186